In [2]:
import sys
sys.executable

'/usr/bin/python'

In [14]:
import os

In [151]:
import pandas as pd
import numpy as np
import random
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
stop_words = set(stopwords.words('english')) 
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/skullscript/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/skullscript/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/skullscript/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/skullscript/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [192]:
from sklearn.datasets import load_files
from sklearn.utils import shuffle
import re

movie_data = load_files(r"./review_polarity/txt_sentoken")
x_data, y_target = movie_data.data, movie_data.target
x, y = shuffle(x_data,y_target, random_state=0)

validate_doc = []
documents = []

num_sent = []
stemmer = WordNetLemmatizer()
for sent in range(0, len(x)):
    sentence = x[sent].split(b"\n")
    wa = sentence
    length = len(sentence)
    num_sent.append(length)

print('Number of Document: ')
print('Min Sentences: ', str(np.min(num_sent) - 1))
print('Max Sentences: ', str(np.max(num_sent) - 1))
print('Avg Sentences: ', str(np.mean(num_sent)- 1))

#preprocessing the data getting rid of numbers special characters etc
total_tok = 0
num_tok = []
collection = []
for sent in range(0, len(x)):
    
    
    #removing special character
    sent_length = len(x[0].split(b"\n"))
    document = re.sub(r'\W', ' ', str(x[sent]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    #print(sent_length)
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    length = len(document)
    num_tok.append(length)
    collection_str = (length/sent_length)
    collection.append(collection_str)
    document = ' '.join(document)
    #print(document)
    documents.append(document)
    #print("------------------------\n")


print('Min Tokens: ',  str(np.min(num_tok)))
print('Max Tokens: ',  str(np.max(num_tok)))
print('Avg Tokens: ',  str(np.mean(num_tok)))
sum_collection = 0;
for i in range(0, len(collection)):
    #print(collection[i])
    sum_collection = sum_collection + collection[i]
print('Avg Sentence Length for Collection: ', str(sum_collection/len(collection)))


Number of Document: 
Min Sentences:  1
Max Sentences:  112
Avg Sentences:  32.36
Min Tokens:  16
Max Tokens:  2305
Avg Tokens:  633.7365
Avg Sentence Length for Collection:  39.60853125


In [189]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

models = [
    RandomForestClassifier(n_estimators=100, random_state=0),
    MultinomialNB(),
    BernoulliNB()
]
countVects = []
tfidfVects = []
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

def cross_validate(modelName, model, data, target):
    print('\n')
    all_accuracies = cross_val_score(estimator=model, X= data, y=target, cv=5)
    print(modelName)
    print("All Accuracies")
    print(all_accuracies)
    print("All Accuracies Mean")
    print(all_accuracies.mean())
    print("Standard Deviation")
    print(all_accuracies.std())
    print('\n')

def validate(models, data, target, train):
    #vectorizer = CountVectorizer(max_features=3000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
    #data_vect = vectorizer.fit_transform(data).toarray()
    #y_pred = models[1].predict(data_vect)
    #print('accuracy score: ' + str(accuracy_score(target, y_pred)))
    
    #vectorizer2 = CountVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
    #data_vect = vectorizer2.fit_transform(data).toarray()
    #y_pred = models[1].predict(data_vect)
    #print('accuracy score: ' + str(accuracy_score(target, y_pred)))
    
    vectorizer3 = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
    data_vect = vectorizer3.fit_transform(data).toarray()
    y_pred = models[1].fit(data_vect)
    y_pred = models[1].predict(data_vect)
    print('accuracy score: ' + str(accuracy_score(target, y_pred)))
    
    #vectorizer4 = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
    #data_vect = vectorizer4.fit_transform(data).toarray()
    #y_pred = models[1].predict(data_vect)
    #print('accuracy score: ' + str(accuracy_score(target, y_pred)))

def model_classification(x,y):
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    X_train, validate_data, Y_train, validate_target = train_test_split(x,y, test_size=0.15, random_state=0)
    x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=0)
    
    # Random Forest Algorithm is used to train our model 
    # The fit method is used to train the algorithm passing training data and training target sets
    from sklearn.model_selection import KFold
    folds = KFold(n_splits=5)
    score = 0;
    
    
    for model in models:
        model_name = ""
        score = 0
        for train_index, test_index in folds.split(x_train,y_train):
            X_train, X_test, Y_train, Y_test = x_train[train_index], x_train[test_index], \
                                           y_train[train_index], y_train[test_index]
            model_name = type(model).__name__
            model.fit(X_train, Y_train)
            y_pred = model.predict(x_test)
            score = score + accuracy_score(y_test, y_pred)
        print(model_name)
        print('avg accuracy fpr 5 fold : ' + str(score/5))
        pred = model.predict(validate_data)
        print(classification_report(validate_target, pred))
        print("Validate Accuracy Score : " + str(accuracy_score(validate_target, pred)))
        print('\n')
        
    
    

def test_diff_feature_sz():
    sizes =[ 500, 1000, 1500, 2000, 2500, 3000 ]
    #test_data, train_data = np.split(documents, [int(0.2*len(x))])
    #test_target, train_target = np.split(y, [int(0.2*len(y))])
    #print(len(test_data))
    #print(len(train_data))
    for sz in sizes:
        
        print('Feature size of ', str(sz) )
        
        print('**********Bag of Words Feature***********')
        vectorizer = CountVectorizer(max_features=sz, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
        x_vect = vectorizer.fit_transform(documents).toarray()
        #x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=0)
        model_classification(x_vect,y)
        
        print('\n')
        print('************* TFIDF Feature **************')
        tfidfconverter = TfidfVectorizer(max_features=sz, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
        x_vect2 = tfidfconverter.fit_transform(documents).toarray()
        #x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=0)
        model_classification(x_vect2,y)

    #validate(models, test_data, test_target, train_data)

test_diff_feature_sz()


Feature size of  500
**********Bag of Words Feature***********
RandomForestClassifier
avg accuracy fpr 5 fold : 0.7629411764705882
              precision    recall  f1-score   support

           0       0.75      0.77      0.76       154
           1       0.75      0.73      0.74       146

    accuracy                           0.75       300
   macro avg       0.75      0.75      0.75       300
weighted avg       0.75      0.75      0.75       300

Validate Accuracy Score : 0.75


MultinomialNB
avg accuracy fpr 5 fold : 0.7641176470588235
              precision    recall  f1-score   support

           0       0.75      0.75      0.75       154
           1       0.73      0.73      0.73       146

    accuracy                           0.74       300
   macro avg       0.74      0.74      0.74       300
weighted avg       0.74      0.74      0.74       300

Validate Accuracy Score : 0.74


BernoulliNB
avg accuracy fpr 5 fold : 0.7588235294117647
              precision    recall

BernoulliNB
avg accuracy fpr 5 fold : 0.798235294117647
              precision    recall  f1-score   support

           0       0.77      0.82      0.80       154
           1       0.80      0.74      0.77       146

    accuracy                           0.78       300
   macro avg       0.78      0.78      0.78       300
weighted avg       0.78      0.78      0.78       300

Validate Accuracy Score : 0.7833333333333333




************* TFIDF Feature **************
RandomForestClassifier
avg accuracy fpr 5 fold : 0.7858823529411764
              precision    recall  f1-score   support

           0       0.78      0.81      0.79       154
           1       0.79      0.75      0.77       146

    accuracy                           0.78       300
   macro avg       0.78      0.78      0.78       300
weighted avg       0.78      0.78      0.78       300

Validate Accuracy Score : 0.7833333333333333


MultinomialNB
avg accuracy fpr 5 fold : 0.8205882352941177
              precision 